In [1]:
import json
from utils.gho_filter import filter_gho_users

with open('results.json', 'r') as fp:
    res = json.load(fp)
data = []
for entry in res:
    data += entry['data']['users']

gho_users = set(filter_gho_users(data))

In [5]:
def get_collats(history):
    return list(set([entry['reserve']['symbol'] for entry in history]))

collaterals = []
for entry in data:
    if entry['id'] in gho_users:
        collaterals += get_collats(entry['supplyHistory'])

# GHO users

In [13]:
import pandas as pd
pd.Series(collaterals).value_counts() / sum(pd.Series(collaterals).value_counts()) * 100

WETH      25.924590
wstETH    13.187804
WBTC      12.375970
USDC      12.285766
AAVE       6.675086
USDT       6.061699
LINK       5.899332
DAI        4.600397
rETH       3.572073
sDAI       1.695833
cbETH      1.280895
CRV        0.865957
LUSD       0.757712
UNI        0.667509
MKR        0.631427
SNX        0.577305
crvUSD     0.414938
           0.360815
RPL        0.342775
LDO        0.324734
BAL        0.288652
ENS        0.234530
USDe       0.180408
FRAX       0.180408
ETHx       0.126285
1INCH      0.108245
PYUSD      0.108245
osETH      0.072163
weETH      0.072163
sUSDe      0.054122
KNC        0.054122
FXS        0.018041
dtype: float64

# Liquidated GHO users

In [16]:
def is_gho_liq(history):
    return any([entry for entry in history if entry['principalReserve']['symbol'] == "GHO"])

gho_liqs = [d for d in data if is_gho_liq(d['liquidationCallHistory'])]
len(gho_liqs)

152

In [20]:
def get_collats(history):
    return list(set([entry['reserve']['symbol'] for entry in history]))

gho_liq_collaterals = []
for entry in gho_liqs:
    gho_liq_collaterals += get_collats(entry['supplyHistory'])

In [21]:
pd.Series(gho_liq_collaterals).value_counts() / sum(pd.Series(gho_liq_collaterals).value_counts()) * 100

WETH      26.730310
wstETH    13.126492
WBTC      10.739857
USDC       9.069212
LINK       8.830549
AAVE       8.591885
USDT       5.727924
DAI        4.057279
rETH       2.863962
cbETH      2.147971
CRV        1.193317
MKR        1.193317
sDAI       0.954654
SNX        0.715990
LUSD       0.715990
BAL        0.715990
ETHx       0.477327
RPL        0.477327
           0.477327
osETH      0.238663
LDO        0.238663
FRAX       0.238663
crvUSD     0.238663
USDe       0.238663
dtype: float64

# V3 Users (non GHO)

In [8]:
non_gho_collaterals = []
for entry in data:
    if entry['id'] not in gho_users:
        non_gho_collaterals += get_collats(entry['supplyHistory'])

In [12]:
pd.Series(non_gho_collaterals).value_counts() / sum(pd.Series(non_gho_collaterals).value_counts()) * 100

WETH      38.714968
USDC      15.069983
USDT      10.862858
WBTC       9.882116
wstETH     6.509596
DAI        4.996265
LINK       3.695645
AAVE       2.279739
rETH       1.430520
CRV        0.821615
cbETH      0.626766
UNI        0.545579
MKR        0.539084
LDO        0.474134
sDAI       0.464391
SNX        0.384828
LUSD       0.378333
crvUSD     0.319878
BAL        0.297145
           0.282532
USDe       0.276037
RPL        0.246809
1INCH      0.151008
ENS        0.141266
FRAX       0.123405
weETH      0.115286
PYUSD      0.110415
sUSDe      0.089306
FXS        0.045465
KNC        0.038970
ETHx       0.037346
osETH      0.025980
STG        0.022732
dtype: float64